# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9b109d2070>
├── 'a' --> tensor([[ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190]])
└── 'x' --> <FastTreeValue 0x7f9b109d7970>
    └── 'c' --> tensor([[ 1.6001,  1.3746, -1.3446, -1.1838],
                        [ 0.1885,  1.4114, -0.5135,  1.2049],
                        [-0.5278, -2.0739,  0.8153, -0.0483]])

In [4]:
t.a

tensor([[ 1.5818, -0.5171, -0.9265],
        [ 0.6563, -0.2178,  0.5190]])

In [5]:
%timeit t.a

62.7 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9b109d2070>
├── 'a' --> tensor([[ 0.7804, -0.6378,  0.4788],
│                   [-1.1535, -0.0718,  0.6785]])
└── 'x' --> <FastTreeValue 0x7f9b109d7970>
    └── 'c' --> tensor([[ 1.6001,  1.3746, -1.3446, -1.1838],
                        [ 0.1885,  1.4114, -0.5135,  1.2049],
                        [-0.5278, -2.0739,  0.8153, -0.0483]])

In [7]:
%timeit t.a = new_value

68.9 ns ± 0.0806 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5818, -0.5171, -0.9265],
               [ 0.6563, -0.2178,  0.5190]]),
    x: Batch(
           c: tensor([[ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5818, -0.5171, -0.9265],
        [ 0.6563, -0.2178,  0.5190]])

In [11]:
%timeit b.a

58 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7505,  1.9907,  0.6866],
               [ 1.3912, -1.0789, -0.6589]]),
    x: Batch(
           c: tensor([[ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

843 ns ± 0.881 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 379 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9b109d7b50>
├── 'a' --> tensor([[[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]],
│           
│                   [[ 1.5818, -0.5171, -0.9265],
│                    [ 0.6563, -0.2178,  0.5190]]])
└── 'x' --> <FastTreeValue 0x7f9b10995220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9a5bd16dc0>
├── 'a' --> tensor([[ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190],
│                   [ 1.5818, -0.5171, -0.9265],
│                   [ 0.6563, -0.2178,  0.5190]])
└── 'x' --> <FastTreeValue 0x7f9a5bd16850>
    └── 'c' --> tensor([[ 1.6001,  1.3746, -1.3446, -1.1838],
                        [ 0.1885,  1.4114, -0.5135,  1.2049],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 29.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.3 µs ± 77.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.6001,  1.3746, -1.3446, -1.1838],
                       [ 0.1885,  1.4114, -0.5135,  1.2049],
                       [-0.5278, -2.0739,  0.8153, -0.0483]],
              
                      [[ 1.6001,  1.3746, -1.3446, -1.1838],
                       [ 0.1885,  1.4114, -0.5135,  1.2049],
                       [-0.5278, -2.0739,  0.8153, -0.0483]],
              
                      [[ 1.6001,  1.3746, -1.3446, -1.1838],
                       [ 0.1885,  1.4114, -0.5135,  1.2049],
                       [-0.5278, -2.0739,  0.8153, -0.0483]],
              
                      [[ 1.6001,  1.3746, -1.3446, -1.1838],
                       [ 0.1885,  1.4114, -0.5135,  1.2049],
                       [-0.5278, -2.0739,  0.8153, -0.0483]],
              
                      [[ 1.6001,  1.3746, -1.3446, -1.1838],
                       [ 0.1885,  1.4114, -0.5135,  1.2049],
                       [-0.5278, -2.0739,  0.8153, -0.0483]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483],
                      [ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483],
                      [ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483],
                      [ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483],
                      [ 1.6001,  1.3746, -1.3446, -1.1838],
                      [ 0.1885,  1.4114, -0.5135,  1.2049],
                      [-0.5278, -2.0739,  0.8153, -0.0483],
                      [ 1.6001,  1.3746, -1.3446, -1.1838],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
